# Data Preparation

Prepare train/val/test splits for coconut tree detection.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kshitijrajsharma/dl4cv-object-detection-on-aerial-imagery/blob/master/notebooks/experiment.ipynb)

In [11]:
# !pip install dl4cv_oda

In [12]:
import requests
import geopandas as gpd
from pathlib import Path
from dl4cv_oda import (clean_osm_data, clip_labels_to_tiles, convert_to_yolo_format,
                       create_train_val_split, create_yolo_config, download_tiles)

In [13]:
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
CHIPS_DIR = DATA_DIR / "chips"
LABELS_DIR = DATA_DIR / "labels"
YOLO_DIR = DATA_DIR / "yolo"

OSM_FILE = RAW_DIR / "kolovai-trees.geojson"
CLEANED_FILE = RAW_DIR / "cleaned.geojson"
TREES_BOX_FILE = DATA_DIR / "trees_box.geojson"
TILES_FILE = DATA_DIR / "tiles.geojson"

if not OSM_FILE.exists():
    OSM_FILE.parent.mkdir(parents=True, exist_ok=True)
    OSM_FILE.write_bytes(requests.get("https://github.com/kshitijrajsharma/dl4cv-oda/blob/master/data/raw/kolovai-trees.geojson?raw=true", allow_redirects=True).content)
    print(f"Downloaded OSM data")

if not CLEANED_FILE.exists():
    count = clean_osm_data(str(OSM_FILE), str(CLEANED_FILE), str(TREES_BOX_FILE))
    print(f"Cleaned {count} trees")

if not TILES_FILE.exists():
    data = gpd.read_file(TREES_BOX_FILE)
    data.to_crs(epsg=4326, inplace=True)
    bbox = list(data.total_bounds)
    await download_tiles(bbox, 19, "https://tiles.openaerialmap.org/5a28639331eff4000c380690/0/5b1b6fb2-5024-4681-a175-9b667174f48c/{z}/{x}/{y}.png", DATA_DIR, 'OAM')
    print("Downloaded tiles")

In [14]:
if not (YOLO_DIR / "train").exists():
    stats = clip_labels_to_tiles(str(TREES_BOX_FILE), str(TILES_FILE), str(LABELS_DIR))
    print(f"Clipped {stats['total_trees']} trees to {stats['processed']} tiles")
    
    class_mapping = convert_to_yolo_format(str(TREES_BOX_FILE), str(CHIPS_DIR), str(LABELS_DIR), str(YOLO_DIR), target_species="Coconut")
    print(f"Converted to YOLO format")
    
    train_count, val_count, test_count = create_train_val_split(str(LABELS_DIR), str(CHIPS_DIR), str(YOLO_DIR), train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, seed=42)
    print(f"Split: train={train_count}, val={val_count}, test={test_count}")
    
    config_file = create_yolo_config(str(YOLO_DIR), {"Coconut": 0})
    print(f"Config: {config_file}")

print("Data preparation complete")

Data preparation complete


# Model Training

In [15]:
import json
from datetime import datetime
import pandas as pd
from ultralytics import YOLO, RTDETR
import torch

## Configuration

In [16]:
RESULTS_DIR = Path("results")
RESULTS_DIR.mkdir(exist_ok=True)

SEED = 42
IMG_SIZE = 256
EPOCHS = 200
PATIENCE = 50
BATCH = 16

MODELS = [
    {"name": "yolov8l", "weights": "yolov8l.pt"},
    {"name": "yolo12l", "weights": "yolo12l.pt"},
    {"name": "rtdetr-l", "weights": "rtdetr-l.pt"},
]

torch.manual_seed(SEED)
exp_id = datetime.now().strftime("%Y%m%d_%H%M%S")
print(f"Experiment: {exp_id}")
print(f"Models: {[m['name'] for m in MODELS]}")

Experiment: 20260111_115533
Models: ['yolov8l', 'yolo12l', 'rtdetr-l']


## Train Models

In [17]:
results = []

for idx, model_cfg in enumerate(MODELS, 1):
    name = model_cfg['name']
    print(f"\n[{idx}/{len(MODELS)}] Training {name}")
    
    model = RTDETR(model_cfg['weights']) if 'rtdetr' in name.lower() else YOLO(model_cfg['weights'])
    
    model.train(
        data=str(YOLO_DIR / "config.yaml"),
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        patience=PATIENCE,
        batch=BATCH,
        seed=SEED,
        name=f"{exp_id}_{name}",
        project="runs",
        plots=True,
        verbose=False
    )
    
    val_metrics = model.val(split='val', verbose=False)
    test_metrics = model.val(split='test', verbose=False)
    
    val_p, val_r = float(val_metrics.box.mp), float(val_metrics.box.mr)
    test_p, test_r = float(test_metrics.box.mp), float(test_metrics.box.mr)
    
    val_f1 = 2 * (val_p * val_r) / (val_p + val_r + 1e-6)
    test_f1 = 2 * (test_p * test_r) / (test_p + test_r + 1e-6)
    
    results.append({
        'model': name,
        'val_precision': val_p,
        'val_recall': val_r,
        'val_f1': val_f1,
        'val_map50': float(val_metrics.box.map50),
        'test_precision': test_p,
        'test_recall': test_r,
        'test_f1': test_f1,
        'test_map50': float(test_metrics.box.map50),
    })
    
    print(f"{name}: val_f1={val_f1:.4f}, test_f1={test_f1:.4f}, test_map50={results[-1]['test_map50']:.4f}")


[1/3] Training yolov8l
New https://pypi.org/project/ultralytics/8.3.252 available 😃 Update with 'pip install -U ultralytics'
New https://pypi.org/project/ultralytics/8.3.252 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.250 🚀 Python-3.13.7 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 15944MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/yolo/config.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_rat

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      1/200      9.09G      1.843       0.49     0.6284        496        256: 100% ━━━━━━━━━━━━ 20/20 5.4it/s 3.7s0.2s

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 11.9it/s 0.3s.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 11.9it/s 0.3s
                   all         93       2846     0.0742      0.354     0.0525     0.0143
                   all         93       2846     0.0742      0.354     0.0525     0.0143

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      2/200         5G      1.635     0.3296      0.374        519        256: 5% ╸─────────── 1/20 1.9it/s 0.3s<10.0s

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      2/200      6.26G      1.149     0.5054     0.2464        790        256: 100% ━━━━━━━━━━━━ 20/20 5.9it/s 3.4s0.2s
      2/200      6.26G      1.149     0.5054     0.2464        790        256: 100% ━━━━━━━━━━━━ 20/20 5.9it/s 3.4s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.6it/s 0.2s.3s
                   all         93       2846      0.617      0.514      0.485       0.16
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.6it/s 0.2s
                   all         93       2846      0.617      0.514      0.485       0.16

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      3/200      6.26G     0.9319     0.5244     0.1862        431        256: 5% ╸─────────── 1/20 2.1it/s 0.3s<9.2s

...

    112/200      7.77G     0.6233      0.481    0.09361        663        256: 100% ━━━━━━━━━━━━ 20/20 5.9it/s 3.4s0.2s
    112/200      7.77G     0.6233      0.481    0.09361        663        256: 100% ━━━━━━━━━━━━ 20/20 5.9it/s 3.4s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.4it/s 0.2s.3s
                   all         93       2846      0.753      0.688       0.65      0.296
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.4it/s 0.2s
                   all         93       2846      0.753      0.688       0.65      0.296

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
    113/200      7.77G     0.7233     0.4684     0.1292        644        256: 5% ╸─────────── 1/20 1.8it/s 0.3s<10.8s

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


    113/200      7.77G     0.6543     0.4915     0.1137        513        256: 100% ━━━━━━━━━━━━ 20/20 6.1it/s 3.3s0.2s
    113/200      7.77G     0.6543     0.4915     0.1137        513        256: 100% ━━━━━━━━━━━━ 20/20 6.1it/s 3.3s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.4it/s 0.2s.3s
                   all         93       2846      0.733      0.676      0.614      0.207
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.4it/s 0.2s
                   all         93       2846      0.733      0.676      0.614      0.207

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
    114/200      7.77G      0.628       0.48     0.1023        506        256: 5% ╸─────────── 1/20 1.8it/s 0.3s<10.5s

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


    114/200      7.77G     0.6207     0.4785    0.09451        658        256: 100% ━━━━━━━━━━━━ 20/20 5.9it/s 3.4s0.2s
    114/200      7.77G     0.6207     0.4785    0.09451        658        256: 100% ━━━━━━━━━━━━ 20/20 5.9it/s 3.4s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.5it/s 0.2s.3s
                   all         93       2846      0.733       0.68      0.626      0.258
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.5it/s 0.2s
                   all         93       2846      0.733       0.68      0.626      0.258
EarlyStopping: Training stopped early as no improvement observed in last 50 epochs. Best results observed at epoch 64, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.
EarlyStopping: Training stopped early as no impro

## Results

In [18]:
df = pd.DataFrame(results)

summary = {
    'exp_id': exp_id,
    'seed': SEED,
    'epochs': EPOCHS,
    'img_size': IMG_SIZE,
    'batch': BATCH,
    'patience': PATIENCE,
    'models': [m['name'] for m in MODELS],
    'results': results,
    'best_model': results[df['test_map50'].idxmax()]['model'] if len(results) > 0 else None,
    'best_test_map50': float(df['test_map50'].max()) if len(results) > 0 else 0.0
}

with open(RESULTS_DIR / f"{exp_id}.json", 'w') as f:
    json.dump(summary, f, indent=2)

print("\nResults")
print(df[['model', 'val_f1', 'val_map50', 'test_f1', 'test_map50']].to_string(index=False))
print(f"\nBest: {summary['best_model']} (test_map50={summary['best_test_map50']:.4f})")
print(f"Saved: results/{exp_id}.json")


Results
   model   val_f1  val_map50  test_f1  test_map50
 yolov8l 0.724576   0.643640 0.763024    0.714543
 yolo12l 0.696003   0.621937 0.744375    0.690536
rtdetr-l 0.740370   0.668512 0.775512    0.729802

Best: rtdetr-l (test_map50=0.7298)
Saved: results/20260111_115533.json


## Analysis

In [20]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for model_cfg in MODELS:
    name = model_cfg['name']
    csv_file = Path(f"runs/{exp_id}_{name}/results.csv")
    
    if csv_file.exists():
        df = pd.read_csv(csv_file)
        df.columns = df.columns.str.strip()
        
        p = df['metrics/precision(B)']
        r = df['metrics/recall(B)']
        f1 = 2 * (p * r) / (p + r + 1e-6)
        
        axes[0].plot(f1, label=name, linewidth=2)
        axes[1].plot(df['metrics/mAP50(B)'], label=name, linewidth=2)

axes[0].set_title('F1 Score')
axes[0].set_ylabel('F1')
axes[0].set_xlabel('Epoch')
axes[0].legend()
axes[0].grid(alpha=0.3)

axes[1].set_title('mAP50')
axes[1].set_ylabel('mAP50')
axes[1].set_xlabel('Epoch')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / f"curves_{exp_id}.png", dpi=150)
plt.show()

<Figure size 1400x500 with 2 Axes>